In [1]:
from spectral import *
import matplotlib.pyplot as plt
import math
import numpy as np
from scipy import constants
import img
import pandas as pd
import os
from joblib import Parallel, delayed
from tqdm import tqdm
import multiprocessing
from scipy.optimize import curve_fit
from scipy.optimize import minimize, Bounds

In [2]:

def load_flux(fluxname):

    with open(fluxname, 'r') as file:
        data = file.readlines()
        #print(data)
        size = len(data)
        #print(size)
        flux = np.zeros(size)
        wavelengths = np.zeros(size)
        for i in range(size):
            flux[i]=(float(data[i].strip().split(" ,")[1]))
            wavelengths[i]=(float(data[i].strip().split(",")[0]))

    return flux,wavelengths



def load_hdr(filename):
    img = envi.open(filename)
    datafile = img.open_memmap(writeable = False)
   
    return datafile


def load_rdn(filename):
    data = np.load(filename)
    return data


def get_metadata(filename):
    img = envi.open(filename)
    img_data = img.open_memmap(writeable = False)
    metadata = envi.read_envi_header(filename)
    metadata_df = pd.DataFrame.from_dict(metadata, orient='index', columns=['Value'])
    band_names = metadata_df.loc['band names', 'Value']
    bandnames = band_names
    reshape_array = img_data.reshape(-1, img_data.shape[-1])
    df = pd.DataFrame(reshape_array)
    df.columns = band_names
    return df


def get_phase_angle(datafile):
    phase_angle = np.empty(datafile.shape[:-1])
   
    for i in range(len(datafile)):
        for j in range(len(datafile[i])):
            for k in  range(len(datafile[i,j])):
                phase_angle[i,j] = datafile[i,j,4]
    phase_angle = phase_angle.reshape((phase_angle.shape[0], phase_angle.shape[1], 1))
   
    return phase_angle

def get_i(datafile):
    angle_i = np.empty(datafile.shape[:-1])
   
    for i in range(len(datafile)):
        for j in range(len(datafile[i])):
            for k in  range(len(datafile[i,j])):
                angle_i[i,j] = datafile[i,j,1]
    angle_i = angle_i.reshape((angle_i.shape[0], angle_i.shape[1], 1))
   
    return angle_i

def get_e(datafile):
    angle_e = np.empty(datafile.shape[:-1])
   
    for i in range(len(datafile)):
        for j in range(len(datafile[i])):
            for k in  range(len(datafile[i,j])):
                angle_e[i,j] = datafile[i,j,3]
    angle_e = angle_e.reshape((angle_e.shape[0], angle_e.shape[1], 1))
   
    return angle_e


def flux_correction(data, flux, angle_i, band):
    corrected_data = np.zeros((data.shape[0],data.shape[1], 1))
    for i in range(len(data)):
        for j in range(len(data[i])):
            corrected_data[i,j,0] = (data[i,j,band]/flux[band]) * (np.pi / np.cos(np.radians(angle_i[i,j,0])))
    corrected_data = corrected_data.reshape(1,-1)
    corrected_data = np.squeeze(corrected_data)  
    return corrected_data



def Data_binning(correcteddata, phase_angle, angle_i, angle_e, bin_size):
   
    phase_angle_ = phase_angle.reshape(1,-1)
    phase_angle_ = np.squeeze(phase_angle_)
    
    phase_i1 = angle_i.reshape(1,-1)
    phase_i1 = np.squeeze(phase_i1)
    
    phase_e1 = angle_e.reshape(1,-1)
    phase_e1 = np.squeeze(phase_e1)
   
    # Define phase angle bin ra# Minimum and maximum phase angle values
    min_angle = np.min(phase_angle)
    max_angle = np.max(phase_angle)
    num_bins = bin_size

    # Calculate the bin width
    bin_width = (max_angle - min_angle) / num_bins

    # Create bin ranges
    bin_ranges = [(min_angle + i * bin_width, min_angle + (i + 1) * bin_width) for i in range(num_bins)]

    binned_reflectance = np.zeros(len(bin_ranges))
    binned_i = np.zeros(len(bin_ranges))
    binned_e = np.zeros(len(bin_ranges))
    
    bin_counts = np.zeros(len(bin_ranges), dtype=int)

    # Bin the reflectance data based on phase angles
    for i, bin_range in enumerate(bin_ranges):
        lower_bound, upper_bound = bin_range
        bin_reflectance = []
        bin_i = []
        bin_e = []
   
        # Loop through phase angles and reflectance values
        for angle1, angle2,angle3, reflectance in zip(phase_angle_,phase_i1,phase_e1, correcteddata):
            if lower_bound <= angle1 < upper_bound:
                bin_reflectance.append(reflectance)
                bin_i.append(angle2)
                bin_e.append(angle3)
   
        # Calculate average reflectance and count
        if bin_reflectance:
            binned_reflectance[i] = np.mean(bin_reflectance)
            bin_counts[i] = len(bin_reflectance)
        
        if bin_i:
            binned_i[i] = np.mean(bin_i)
            bin_counts[i] = len(bin_i)
        
        if bin_e:
            binned_e[i] = np.mean(bin_e)
            bin_counts[i] = len(bin_e)
   
    binned_phase_angle = []
    for bin_tuple in bin_ranges:
        average = np.mean(bin_tuple)
        binned_phase_angle.append(average)
  
    return binned_phase_angle,binned_i,binned_e ,binned_reflectance

def phase_plot(binned_phase_angle,binned_reflectance, band):
    plt.scatter(binned_phase_angle,binned_reflectance,s=10)
    plt.xlabel('Phase angle(binned and averaged)')
    plt.ylabel('Reflectance(Averaged/binned)')
    plt.title(f'Spectrum for {band}th band')

def incident_plot(binned_i,binned_reflectance, band):
    plt.scatter(binned_i,binned_reflectance,s=10)
    plt.xlabel('Incident angle(binned)')
    plt.ylabel('Reflectance(Averaged/binned)')
    plt.title(f'Spectrum for {band}th band')

def emission_plot(binned_e,binned_reflectance, band):
    plt.scatter(binned_e,binned_reflectance,s=10)
    plt.xlabel('Emission angle(binned)')
    plt.ylabel('Reflectance(Averaged/binned)')
    plt.title(f'Spectrum for {band}th band')


def spectral_phase_angle_analysis(fluxdatafile, imgdatafile, rdndatafile, band, binsize):
    flux, wavelengths = load_flux(fluxdatafile)
    img_data = load_hdr(imgdatafile)
    rdn_data = load_rdn(rdndatafile)
    phaseangle_data = get_phase_angle(img_data)
    i_data = get_i(img_data)
    e_data = get_e(img_data)
    corr_data = flux_correction(rdn_data, flux,band)
    phase,i_angle,e_angle, binned_reflect = Data_binning(corr_data, phaseangle_data,i_data,e_data, binsize)
    #phase_plot(phase,binned_reflect,band)

    return phase, i_angle,e_angle ,binned_reflect

In [3]:
flux,wavelengths = load_flux('/home/ritabik/Desktop/Guneshwar/m3_solarflux.txt')
img1 = load_hdr('/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20081129t052459_v03_obs.hdr')
radiance1 = load_rdn('/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20081129t052459_v03_rdn.npy')
i_data1 = get_i(img1)
e_data1 = get_e(img1)
phase1 = get_phase_angle(img1)
corr_data1 = flux_correction(radiance1,flux,i_data1,50)


In [4]:
img2 = load_hdr('/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20081129t171431_v03_obs.hdr')
radiance2 = load_rdn('/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20081129t171431_v03_rdn.npy')
i_data2 = get_i(img2)
e_data2 = get_e(img2)
phase2 = get_phase_angle(img2)
corr_data2 = flux_correction(radiance2,flux,i_data2,50)

In [5]:
img3 = load_hdr('/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20081129t171509_v03_obs.hdr')
radiance3 = load_rdn('/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20081129t171509_v03_rdn.npy')
i_data3 = get_i(img3)
e_data3 = get_e(img3)
phase3 = get_phase_angle(img3)
corr_data3 = flux_correction(radiance3,flux,i_data3,50)


In [6]:
img4 = load_hdr('/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20081130t050351_v03_obs.hdr')
radiance4 = load_rdn('/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20081130t050351_v03_rdn.npy')
i_data4 = get_i(img4)
e_data4 = get_e(img4)
phase4 = get_phase_angle(img4)
corr_data4 = flux_correction(radiance4,flux,i_data4,50)


KeyboardInterrupt: 

In [ ]:
img5 = load_hdr('/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20081130t165323_v03_obs.hdr')
radiance5 = load_rdn('/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20081130t165323_v03_rdn.npy')
i_data5 = get_i(img5)
e_data5 = get_e(img5)
phase5 = get_phase_angle(img5)
corr_data5 = flux_correction(radiance5,flux,i_data5,50)


In [ ]:
img6 = load_hdr('/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090123t064545_v03_obs.hdr')
radiance6 = load_rdn('/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090123t064545_v03_rdn-017.npy')
i_data6 = get_i(img6)
e_data6 = get_e(img6)
phase6 = get_phase_angle(img6)
corr_data6 = flux_correction(radiance6,flux,i_data6,50)


In [ ]:
img7 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090124t062145_v03_obs.hdr'
radiance7 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090124t062145_v03_rdn-007.npy'
x7,y7,z7,w7 = spectral_phase_angle_analysis(flux, img7, radiance7, 50,100)

In [ ]:
img8 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090622t082242_v03_obs.hdr'
radiance8 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090622t082242_v03_rdn-003.npy'
x8,y8,z8,w8 = spectral_phase_angle_analysis(flux, img8, radiance8, 50,100)

In [ ]:
img9 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090622t124051_v03_obs.hdr'
radiance9 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090622t124051_v03_rdn-002.npy'
x9,y9,z9,w9 = spectral_phase_angle_analysis(flux, img9, radiance9, 50,100)

In [ ]:
img10 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090623t012541_v03_obs.hdr'
radiance10 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090623t012541_v03_rdn-015.npy'
x10,y10,z10,w10 = spectral_phase_angle_analysis(flux, img10, radiance10, 50,100)

In [ ]:
img11 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090623t052831_v03_obs.hdr'
radiance11 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090623t052831_v03_rdn-006.npy'
x11,y11,z11,w11 = spectral_phase_angle_analysis(flux, img11, radiance11, 50,100)

In [ ]:
img12 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090623t095551_v03_obs.hdr'
radiance12 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090623t095551_v03_rdn-012.npy'
x12,y12,z12,w12 = spectral_phase_angle_analysis(flux, img12, radiance12, 50,100)

In [ ]:
img13 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090623t135841_v03_obs.hdr'
radiance13 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090623t135841_v03_rdn-011.npy'
x13,y13,z13,w13 = spectral_phase_angle_analysis(flux, img13, radiance13, 50,100)

In [ ]:
img14 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090623t142528_v03_obs.hdr'
radiance14 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090623t142528_v03_rdn.npy'
x14,y14,z14,w14 = spectral_phase_angle_analysis(flux, img14, radiance14, 50,100)

In [ ]:
img15 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090623t182551_v03_obs.hdr'
radiance15 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090623t182551_v03_rdn-010.npy'
x15,y15,z15,w15 = spectral_phase_angle_analysis(flux, img15, radiance15, 50,100)

In [ ]:
img16 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090719t160300_v03_obs.hdr'
radiance16 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090719t160300_v03_rdn-016.npy'
x16,y16,z16,w16 = spectral_phase_angle_analysis(flux, img16, radiance16, 50,100)

In [ ]:
img17 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090719t200620_v03_obs.hdr'
radiance17 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090719t200620_v03_rdn-005.npy'
x17,y17,z17,w17 = spectral_phase_angle_analysis(flux, img17, radiance17, 50,100)

In [ ]:
img18 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090719t203200_v03_obs.hdr'
radiance18 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090719t203200_v03_rdn-004.npy'
x18,y18,z18,w18 = spectral_phase_angle_analysis(flux, img18, radiance18, 50,100)

In [ ]:
img19 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090720t003411_v03_obs.hdr'
radiance19 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090720t003411_v03_rdn(1)-020.npy'
x19,y19,z19,w19 = spectral_phase_angle_analysis(flux, img19, radiance19, 50,100)

In [ ]:
img20 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090720t043741_v03_obs.hdr'
radiance20 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090720t043741_v03_rdn-019.npy'
x20,y20,z20,w20 = spectral_phase_angle_analysis(flux, img20, radiance20, 50,100)

In [ ]:
img21 = '/home/ritabik/Desktop/Guneshwar/Observation_data/m3g20090720t090521_v03_obs.hdr'
radiance21 = '/home/ritabik/Desktop/Guneshwar/radiance_data/corr_i_m3g20090720t090521_v03_rdn-018.npy'
x21,y21,z21,w21 = spectral_phase_angle_analysis(flux, img21, radiance21, 50,100)

In [ ]:
x = np.concatenate([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21])
y = np.concatenate([y1, y2, y3, y4, y5, y6, y7, y8, y9, y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21])
z = np.concatenate([z1, z2, z3, z4, z5, z6, z7, z8, z9, z10,z11,z12,z13,z14,z15,z16,z17,z18,z19,z20,z21])
w = np.concatenate([w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18, w19, w20, w21])

In [ ]:
plt.scatter(x,w,alpha=0.5)
plt.show()

In [ ]:
sorted_indices = np.argsort(x)

# Apply the sorted indices to all three arrays
x_sorted = x[sorted_indices]
y_sorted = y[sorted_indices]
z_sorted = z[sorted_indices]
w_sorted = w[sorted_indices]



In [ ]:
plt.scatter(x_sorted,w_sorted,alpha=0.5)
plt.xlabel('Phase(in deg)')
plt.ylabel('Reflectance')
plt.title('Reflectance spectra of all 21 images')
plt.show()

In [ ]:
np.savetxt('data1.txt', np.column_stack((x, y, z,w)), header='phase cosi reflectance', fmt='%.6f', delimiter='\t')

In [ ]:
def p(g, b):
    term1 = ((1 + c(g, b)) / 2) * ((1 - b**2) / (1 - 2 * b * np.cos(g) + b**2)**(3/2))
    term2 = ((1 - c(g, b)) / 2) * ((1 - b**2) / (1 + 2 * b * np.cos(g) + b**2)**(3/2))
    return term1 + term2


def c(g, b):
    return 3.29 * np.exp(-17.4 * b**2) - 0.98

def Bs(g, hs):
    return 1 / (1 + (np.tan(g/2))/hs)

def M(i, e, w):
    return H(np.cos(i), w) * H(np.cos(e), w) - 1

def H(x, w):
    r0 = (1 - np.sqrt(1 - w)) / (1 + np.sqrt(1 - w))
    return (1 - w * x*(r0 + ((1 - 2 * r0 * x)/2) * np.log((1 + x )/x)))**(-1)


i = (y_sorted*np.pi)/180
g = (x_sorted*np.pi)/180
e = (z_sorted*np.pi)/180
y_observed = w_sorted

def RADF(g, w, b, hs, B_so):
    i = (y_sorted*np.pi)/180
    e = (z_sorted*np.pi)/180
    mu = np.cos(e)
    mu0 = np.cos(i)
    results1 = (w / 4) * (mu0 / (mu0 + mu)) * (p(g, b) * (1 + B_so * Bs(g, hs)) + M(i, e, w))
    
    mean_value = np.nanmean(results1)
    results = np.where(np.isnan(results1), mean_value, results1)
    
    return results

initial_guess = [0.2, 0.6, 0.07, 2.4]  # Initial guess for [w, b, hs, B_so]

# Bounds for parameters
bounds = ([0.15, 0, 0.06, 1.6], [0.45, 1, 0.08, 2.6])  # No bounds, adjust according to your problem
# Call curve_fit with g as the independent variable
fitted_params, covariance = curve_fit(RADF, g, y_observed, initial_guess, bounds=bounds)

print(fitted_params)
# Plot the true data and the fitted model
plt.figure(figsize=(8, 6))
plt.scatter(g, y_observed, label='Observed Data')
plt.plot(g, RADF(g, *fitted_params), color='red', label='Fitted Model',alpha=0.5)
plt.xlabel('Phase(in radians)')
plt.ylabel('Reflectance')
plt.title('Reflectance Spectra')
plt.xlim(0,1.6)
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

i = (y_sorted*np.pi)/180
g = (x_sorted*np.pi)/180
e = (z_sorted*np.pi)/180
y_observed = w_sorted

# Define the functions for the model
def p(g, b):
    term1 = ((1 + c(g, b)) / 2) * ((1 - b**2) / (1 - 2 * b * np.cos(g) + b**2)**(3/2))
    term2 = ((1 - c(g, b)) / 2) * ((1 - b**2) / (1 + 2 * b * np.cos(g) + b**2)**(3/2))
    return term1 + term2

def c(g, b):
    return 3.29 * np.exp(-17.4 * b**2) - 0.98

def Bs(g, hs):
    return 1 / (1 + (np.tan(g/2))/hs)

def M(i, e, w):
    return H(np.cos(i), w) * H(np.cos(e), w) - 1

def H(x, w):
    r0 = (1 - np.sqrt(1 - w)) / (1 + np.sqrt(1 - w))
    return (1 - w * x*(r0 + ((1 - 2 * r0 * x)/2) * np.log((1 + x )/x)))**(-1)

# Define the model function to fit
def RADF(g, w, b, hs, B_so):
    i = (y_sorted*np.pi)/180
    e = (z_sorted*np.pi)/180
    mu = np.cos(e)
    mu0 = np.cos(i)
    results1 = (w / 4) * (mu0 / (mu0 + mu)) * (p(g, b) * (1 + B_so * Bs(g, hs)) + M(i, e, w))
    
    mean_value = np.nanmean(results1)
    results = np.where(np.isnan(results1), mean_value, results1)
    
    return results

# Define the objective function (chi-square)
def chi_square(params, g, y_observed):
    y_predicted = RADF(g, *params)
    residuals = y_observed - y_predicted
    variance = np.var(y_observed)  # Compute the variance of the observed data
    return np.sum(residuals ** 2)

# Load the observed data
# Assuming y_sorted, x_sorted, and z_sorted are already defined

# Initial guess for parameters
params = np.array([0.2, 0.5, 0.05, 2.4])

# Bounds for parameters
lower_bounds = np.array([0.15, 0, 0.06, 1.6])
upper_bounds = np.array([0.45, 1, 0.08, 2.6])

def jacobian(params, g):
    step = 1e-6
    num_params = len(params)
    num_points = len(g)
    jac = np.zeros((num_points, num_params))
    for i in range(num_params):
        params_perturbed = np.array(params)
        params_perturbed[i] += step
        jac[:, i] = (RADF(g, *params_perturbed) - RADF(g, *params)) / step
    return jac

# Perform the LM algorithm with parameter bounds
max_iterations = 10000
tolerance = 1e-6
lambda_val = 0.0001
for _ in range(max_iterations):
    residuals = y_observed - RADF(g, *params)
    jac = jacobian(params, g)
    grad = jac.T.dot(residuals)
    hessian = jac.T.dot(jac)
    hessian += lambda_val * np.diag(np.diag(hessian))  # Regularization
    delta_params = np.linalg.solve(hessian, grad)
    
    # Apply bounds to delta_params
    delta_params = np.maximum(np.minimum(delta_params, upper_bounds - params), lower_bounds - params)
    
    params_new = params + delta_params
    if np.linalg.norm(delta_params) < tolerance:
        print("LM algorithm converged.")
        break
    if chi_square(params_new, g, y_observed) < chi_square(params, g, y_observed):
        lambda_val /= 10
        params = params_new
    else:
        lambda_val *= 10

# Print the fitted parameters
print("Fitted Parameters:", params)

# Plot the observed data and the fitted model
plt.figure(figsize=(8, 6))
plt.scatter(g, y_observed, label='Observed Data')
plt.plot(g, RADF(g, *params), color='red', label='Fitted Model')
plt.xlabel('To-Sun Azimuth (deg)')
plt.ylabel('Reflectance')
plt.title('Fitting Model with LM Algorithm and Bounds')
plt.legend()
plt.grid(True)
plt.xlim(0,1.6)
plt.show()
